In [ ]:
###################################################
#=====================591 crawler==================
###################################################
from PIL import Image
import requests 
from bs4 import BeautifulSoup
import json 
from io import BytesIO
import pytesseract
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import cluster, datasets, metrics
from pandas.tools.plotting import table
from collections import Counter


a={}
temp = {}
http = []
detail = []
phone = []
url_phone = []
df = pd.DataFrame()
data = {}
make_to_dict = []
result = pd.DataFrame()

headers = { 
    'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36','Accept-Language':'zh-TW'
}
#==================API of each page================
url = ['https://business.591.com.tw/home/search/rsList?is_new_list=1&storeType=1&type=1&kind=5&searchtype=1&region=1&firstRow={}&totalRows=2833'.format(str(i)) for i in range(0,2826,30)]
for m in range(0,95):
    res = requests.get(url[m], headers = headers)
    print("url[m]=",m)
    data[m] = json.loads(res.text)
#====================phone number==================
def get_text(image):
    return pytesseract.image_to_string(image) #get number which is in the image
#===================house information==============
for i in range(0,95):
    for j in range(0,30):
        temp = 'https://rent.591.com.tw/rent-detail-' + str(data[i]['data']['data'][j]['id']) + '.html' #each link
        #print("i=",i,"j=",j)
        #print(temp)
        #http[i].append(temp)        
        #print('https://rent.591.com.tw/' + data[i]['data']['topData'][j]['detail_url'])
        respond = requests.get(temp,headers = headers)
        soup = BeautifulSoup(respond.text,'html.parser')
        article_name = 'h1 span.houseInfoTitle'
        household_name = 'div.avatarRight div'
        image_name = 'div.j-phone.infoTwo.clearfix-new.phone-hide span.num img'
        ping_name = 'ul.attr li'
        price_name = 'div.price.clearfix i'
        floor_name = 'ul.attr li'
        decorate_name = 'div.two em'
        address_name = 'div.clearfix span.addr'
        
        article = soup.select(article_name)
        household = soup.select(household_name)
        ping = soup.select(ping_name)
        phone_image = soup.select(image_name)
        price = soup.select(price_name)
        floor = soup.select(floor_name)
        decorate = soup.select(decorate_name)
        address = soup.select(address_name)
        if not phone_image:
            image_name = 'div.j-phone.infoTwo.clearfix-new.phone-hide span.num'
            phone_image = soup.select(image_name)
            if not phone_image:
                continue #the object is removed
            else:
                temp3 = phone_image[0].text
        else:
            url_phone = 'http:' + phone_image[0]['src']
            temp2 = requests.get(url_phone, headers = headers, stream=True, verify=False)
            b = Image.open(BytesIO(temp2.content))
            pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files (x86)\\Tesseract-OCR\\tesseract.exe'
            temp3 = pytesseract.image_to_string(b)
        print(article[0].text)
        #print(household[0].text)
        #print(address[0].text)
        #print(address[0].text[3:6])
        #print(temp3)
        #print(price[0].text)
        #print(ping[0].text)
        #print(floor[1].text)
        
        #for k in decorate:
        #    print(k['title'])
        make_to_dict = {'article':article[0].text,'household':household[0].text,"address":address[0].text,"area":address[0].text[3:6],
                        "phone":temp3,"price":price[0].text,"space":ping[0].text,'management_fee':decorate[1].text,
                        'parking_space':decorate[2].text,'lease_tern':decorate[3].text,'decorate':decorate[4].text}
                        
        df = df.append(make_to_dict,ignore_index=True)
        print("========================")
df.to_csv('C:\\Users\\user\\Desktop\\cathy5.csv',index=False)
#df = pd.read_csv('C:\\Users\\user\\Desktop\\cathy5.csv',encoding='big5')


In [ ]:
####################################################
#===============data processing=====================
####################################################
df.iloc[list(np.where(df.iloc[:,[1]]=="中正區")[0]),[1]] = str("1")
df.iloc[list(np.where(df.iloc[:,[1]]=="中山區")[0]),[1]] = str("2")
df.iloc[list(np.where(df.iloc[:,[1]]=="大安區")[0]),[1]] = str("3")
df.iloc[list(np.where(df.iloc[:,[1]]=="信義區")[0]),[1]] = str("4")
df.iloc[list(np.where(df.iloc[:,[1]]=="士林區")[0]),[1]] = str("5")
df.iloc[list(np.where(df.iloc[:,[1]]=="內湖區")[0]),[1]] = str("6")
df.iloc[list(np.where(df.iloc[:,[1]]=="文山區")[0]),[1]] = str("7")
df.iloc[list(np.where(df.iloc[:,[1]]=="北投區")[0]),[1]] = str("8")
df.iloc[list(np.where(df.iloc[:,[1]]=="松山區")[0]),[1]] = str("9")
df.iloc[list(np.where(df.iloc[:,[1]]=="南港區")[0]),[1]] = str("10")
df.iloc[list(np.where(df.iloc[:,[1]]=="大同區")[0]),[1]] = str("11")
df.iloc[list(np.where(df.iloc[:,[1]]=="萬華區")[0]),[1]] = str("12")

#============find the identity=======================
df.iloc[list(np.where(df['household'].str.find("屋主")>1)[0]),[4]] = str("2")
df.iloc[list(np.where(df['household'].str.find("仲介")>1)[0]),[4]] = str("1")
df.iloc[list(np.where(df['household'].str.find("代理人")>1)[0]),[4]] = str("0")

#==================decorate==========================
#np.unique(df.iloc[:,[3]]) # get all element
df.iloc[list(np.where(df['decorate'].str.find("已辦"))[0]),[3]] = str("0")
df.iloc[list(np.where(df['decorate'].str.find("不提供"))[0]),[3]] = str("0")
df.iloc[list(np.where(df['decorate'].str.find("--"))[0]),[3]] = str("0")
df.iloc[list(np.where(df['decorate'].str.find("提供"))[0]),[3]] = str("0")
df.iloc[list(np.where(df['decorate'].str.find("未辦"))[0]),[3]] = str("0")
df.iloc[list(np.where(df['decorate'].str.find("尚未裝潢"))[0]),[3]] = str("0")
df.iloc[list(np.where(df['decorate'].str.find("簡易裝潢"))[0]),[3]] = str("1")
df.iloc[list(np.where(df['decorate'].str.find("中檔裝潢"))[0]),[3]] = str("2")
df.iloc[list(np.where(df['decorate'].str.find("高檔裝潢"))[0]),[3]] = str("3")

#==============lease tern===========================
list(np.unique(str(df.iloc[:,[5]]))) # get all element
df.iloc[list(np.where(df.iloc[:,[5]]=="15天")[0]),[5]] = float(0.04)
df.iloc[list(np.where(df.iloc[:,[5]]=="11年")[0]),[5]] = int(11)
df.iloc[list(np.where(df.iloc[:,[5]]=="一年")[0]),[5]] = int(1)
df.iloc[list(np.where(df.iloc[:,[5]]=="1年")[0]),[5]] = int(1)
df.iloc[list(np.where(df.iloc[:,[5]]=="10天")[0]),[5]] = float(0.0274)
df.iloc[list(np.where(df.iloc[:,[5]]=="1天")[0]),[5]] = float(0.0027)
df.iloc[list(np.where(df.iloc[:,[5]]=="1月")[0]),[5]] = float(0.083)
df.iloc[list(np.where(df.iloc[:,[5]]=="2年")[0]),[5]] = int(2)
df.iloc[list(np.where(df.iloc[:,[5]]=="3年")[0]),[5]] = int(3)
df.iloc[list(np.where(df.iloc[:,[5]]=="3月")[0]),[5]] = float(0.25)
df.iloc[list(np.where(df.iloc[:,[5]]=="4年")[0]),[5]] = int(4)
df.iloc[list(np.where(df.iloc[:,[5]]=="5年")[0]),[5]] = int(5)
df.iloc[list(np.where(df.iloc[:,[5]]=="三個月")[0]),[5]] = int(5)
df.iloc[list(np.where(df.iloc[:,[5]]=="不提供")[0]),[5]] = int(0)
df.iloc[list(np.where(df.iloc[:,[5]]=="兩年")[0]),[5]] = int(0)
df.iloc[list(np.where(df.iloc[:,[5]]=="半年")[0]),[5]] = int(0.5)
df.iloc[list(np.where(df.iloc[:,[5]]=="提供")[0]),[5]] = int(0)

#===============management fee======================
np.unique(df.iloc[:,[6]]) # get all element

df['management_fee'] = df['management_fee'].str.replace('元/月','')
df.iloc[list(np.where(df.iloc[:,[6]]=="無")[0]),[6]] = int(0)
df.iloc[list(np.where(df.iloc[:,[6]]=="--")[0]),[6]] = int(0)
df.iloc[list(np.where(df.iloc[:,[6]]=="1月")[0]),[6]] = int(0)
df.iloc[list(np.where(df.iloc[:,[6]]=="一年")[0]),[6]] = int(0)
df[['management_fee']] = df[['management_fee']].astype(float)

#===================price===========================
list(np.unique(df.iloc[:,[7]])) # get all element
df.iloc[list(np.where(df.iloc[:,[7]]=="一年")[0]),[1]] = int(0)
#df.iloc[list(np.where(df.iloc[:,[7]] in "/月")[0]),[1]]
#np.where(df.iloc[:,[10]]) in "/月"
df['price'] = df['price'].str.replace('元/月','')
df['price'] = df['price'].str.replace(',','')
#df['price'].str.contains('/月')# get'/月'

#===================space===========================
df['space'] = df['space'].apply(lambda x:x[6:])
df['space'] = df['space'].apply(lambda x:x[:-1])

#===============parking space=======================
list(np.unique(df.iloc[:,[7]])) # get all element
df.iloc[list(np.where(df.iloc[:,[7]]=="無")[0]),[7]] = int(0)
df.iloc[list(np.where(df.iloc[:,[7]]=="0元/月")[0]),[7]] = int(0)
df.iloc[list(np.where(df.iloc[:,[7]]=="元/月")[0]),[7]] = int(0)
df.iloc[list(np.where(df.iloc[:,[7]]=="平面式停車位，費用另計")[0]),[7]] = int(1)
df.iloc[list(np.where(df.iloc[:,[7]]=="機械式停車位，費用另計")[0]),[7]] = int(1)
df.iloc[list(np.where(df.iloc[:,[7]]=="平面式停車位，已含租金內")[0]),[7]] = int(2)
df.iloc[list(np.where(df.iloc[:,[7]]=="機械式停車位，已含租金內")[0]),[7]] = int(2)
df['parking_space'].astype(int)



In [ ]:
####################################################
#=================analysis==========================
####################################################
each_man_house_number = df.groupby('phone').count().iloc[:,[1]]
df = pd.merge(df,each_man_house_number,left_on='phone', right_index=True)
df['price'] = df['price'].astype(int)
df = df.rename(columns={'area_y': 'house_number'}) #rename
household_house_number = df.drop_duplicates(['phone'])['house_number'] #each household or realtor house number 
plt.hist(household_house_number)
Counter(household_house_number) #show the table of household_house_number
#household_house_number.describe() #summary household_house_number
#df['price'].type
person_price = df.groupby('phone').agg({'price':'sum'}) #each person house price
person_price_list = list(person_price['price'])
plt.plot(person_price_list) #according to price, we can find abnormal data is data 1206, the price is too high
df = df.drop(df.index[np.where(df.iloc[:,[9]] == person_price.max())[0][0]]) #so erase it
person_price = df.groupby('phone').agg({'price':'sum'}) #each person house price
df = pd.merge(df,person_price,left_on='phone', right_index=True)
person_price_list = list(person_price['price'])
plt.plot(person_price_list) #each person get money/month
plt.hist(person_price_list,range=(10000, 1000000)) #display the passive income
for_household = df.iloc[list(np.where(df.iloc[:,[4]]=="1") or np.where(df.iloc[:,[4]]=="0"))[0],:]
temp_1 = list(np.where(df.iloc[:,[4]]=="1"))[0] 
temp_2 = list(np.where(df.iloc[:,[4]]=="0"))[0]
for_household = np.append(temp_1,temp_2)
for_household = df.iloc[for_household,:]
plt.hist(for_household['price_y'],range=(50000, 3000000))
owner = df.iloc[list(np.where(df.iloc[:,[4]]=="2"))[0],:]
plt.hist(owner['price_y'],range=(50000, 200000))
owner['price_y'].describe()


In [ ]:
####################################################
#=================clustering========================
####################################################
from sklearn import cluster, metrics
from sklearn.cluster import KMeans
x = df.iloc[:,[1,3,4,5,6,7,9,10]]
silhouette_avgs = []
ks = range(2, 11)
for k in ks:
    kmeans_fit = cluster.KMeans(n_clusters = k).fit(df.iloc[:,[1,3,4,5,6,7,9,10]])
    cluster_labels = kmeans_fit.labels_
    silhouette_avg = metrics.silhouette_score(df.iloc[:,[1,3,4,5,6,7,9,10]], cluster_labels)
    silhouette_avgs.append(silhouette_avg)
#plot from 2 to 10 clustering
plt.bar(ks, silhouette_avgs)
plt.show()
print(silhouette_avgs)

km = KMeans(n_clusters=2)  #K=2
y_pred = km.fit_predict(df.iloc[:,[1,3,4,5,6,7,9,10]])
plt.figure(figsize=(10, 6))
plt.xlabel('Salary')
plt.ylabel('Rate of working')
plt.scatter(x.iloc[:, 0], x.iloc[:, 1], c=y_pred) #C是第三維度 已顏色做維度
plt.show()